In [1]:
import pandas as pd
import numpy as np
import re

raw = pd.read_csv('./bioinf-edit1 - All.csv')

In [2]:
# Mating pathway
mating_pathway = {
    "Ste2",
    "Ste3",
    "Gαβγ",
    "Ste5",
    "Ste4","Ste1","Gpa1",
    "Cdc24",
    "Cdc42",
    "Ste20",
    "Ste50",
    "Ste11",
    "Ste7",
    "Fus3",
    "Ste12",
}

# Filamentous growth pathway
filamentous_growth_pathway = {
    "Msb2",
    "Sho1","Opy2",
    "Cdc24",
    "Cdc42",
    "Ste20",
    "Ste50",
    "Ste11",
    "Ste7",
    "Kss1",
    "Ste12","Tec1",
}

# High osmolarity pathway
high_osmolarity_pathway = {
    "Msb2","Hkr1",
    "Sho1","Opy2",
    "Cdc24",
    "Cdc42",
    "Ste20",
    "Ste50",
    "Ste11",
    "Pbs2",
    "Hog1",
    "Hot1",
}

# Combine into a dictionary for easy access
pathways_hm = {
    "Mating": mating_pathway,
    "Filamentous Growth": filamentous_growth_pathway,
    "High Osmolarity": high_osmolarity_pathway
}
pathways = set([x.lower() for x in mating_pathway.union(high_osmolarity_pathway).union(filamentous_growth_pathway)])

In [3]:
def extract_gene_name(info_field):
    if "ANN=" in info_field:
        ann_entries = info_field.split("ANN=")[1].split(";")[0]
        annotations = ann_entries.split(",")
        gene_info = set()
        for ann in annotations:
            ann_fields = ann.split("|")
            if len(ann_fields) > 4:
                gene_name = ann_fields[3]
                #gene_id = ann_fields[4]
                gene_info.add(gene_name.lower())
        return gene_info
    else:
        return set() 

def augument_data(data):
  data = data.replace('./.:.',  '')
  sorting_table = {"HIGH":0, "MODERATE":1, "LOW":2, 'MODIFIER':3}
  data['Severity'] = data['FORMAT'].apply(lambda x: sorting_table[x.split('|')[2]])
  data['GENES_AFFECTED'] = data['FORMAT'].map(extract_gene_name)
  #data['Gene'] = data['FORMAT'].apply(lambda x: x.split('|')[3])
  return data

data = augument_data(raw)

strains = list(map(lambda x : x+'.bam', ['G28c1', 'D11c1', 'I38c2', 'M59c1', 'WT']))

In [11]:
def analysis(strain, quality_threshold = 100 ):
  is_homozygous = lambda x: x.startswith('1/1')
  others = strains.copy() # strains = list(map(lambda x : x+'.bam', ['G28c1', 'D11c1', 'I38c2', 'M59c1', 'WT']))
  others.remove(strain)
  promising_picks = data[
    (data[strain].apply(is_homozygous)) &
    (data["QUAL"] > quality_threshold) &
    (data[others].applymap(is_homozygous).sum(axis=1)<=2)].sort_values(by='Severity', ascending=True)
  pcks = set()
  for ga in promising_picks['GENES_AFFECTED']: pcks = pcks.union(ga)
  cross = pathways.intersection(pcks)
  if len(cross) > 0:
    print(f'FOUND cross between pathway and affected gene in {strain}\n\t{cross}')
  else:
    print(f'FOUND nothing in {strain}')
  return promising_picks


In [12]:
for strain in strains:
  analysis(strain)

FOUND nothing in G28c1.bam
FOUND cross between pathway and affected gene in D11c1.bam
	{'ste12'}
FOUND cross between pathway and affected gene in I38c2.bam
	{'kss1'}
FOUND nothing in M59c1.bam
FOUND nothing in WT.bam


In [6]:
pathways

{'cdc24',
 'cdc42',
 'fus3',
 'gpa1',
 'gαβγ',
 'hkr1',
 'hog1',
 'hot1',
 'kss1',
 'msb2',
 'opy2',
 'pbs2',
 'sho1',
 'ste1',
 'ste11',
 'ste12',
 'ste2',
 'ste20',
 'ste3',
 'ste4',
 'ste5',
 'ste50',
 'ste7',
 'tec1'}

In [8]:
s = 3
print(strains[s])
analysis(strains[s], quality_threshold=60)

M59c1.bam
FOUND nothing in M59c1.bam


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Unnamed: 9,D11c1.bam,G28c1.bam,I38c2.bam,M59c1.bam,WT.bam,Severity,GENES_AFFECTED
370,XIV,668562,.,C,A,221.9990,.,VDB=0.43412;SGB=-0.69312;MQSB=1;MQ0F=0;AF1=1;A...,ANN=A|stop_gained|HIGH|YNR021W|YNR021W|transcr...,GT:PL,,,,"1/1:255,96,0",,0,{ynr021w}
389,XV,563576,.,G,T,221.9990,.,VDB=0.552091;SGB=-0.693141;MQSB=1;MQ0F=0;AF1=1...,ANN=T|stop_gained|HIGH|RGA1|YOR127W|transcript...,GT:PL,,,,"1/1:255,111,0",,0,{rga1}
437,Mito,65922,.,GGGGGC,G,214.4580,.,INDEL;IDV=1;IMF=0.00571429;VDB=1.50203e-08;SGB...,ANN=G|frameshift_variant|HIGH|Q0182|Q0182|tran...,GT:PL,,,,"1/1:255,255,0",,0,{q0182}
19,I,12169,.,C,T,77.0126,.,VDB=0.182083;SGB=-0.616816;RPB=0.931779;MQB=0....,ANN=T|missense_variant|MODERATE|YAL064W-B|YAL0...,GT:PL,"0/1:87,0,4",,,"1/1:107,3,0",,1,{yal064w-b}
299,XI,479905,.,T,A,221.9990,.,VDB=0.00732327;SGB=-0.691153;MQSB=1;MQ0F=0;AF1...,ANN=A|missense_variant|MODERATE|ALY1|YKR021W|t...,GT:PL,,,,"1/1:255,54,0",,1,{aly1}
